# Simulating a FHN network.

## Set parameters and define the network

In [22]:
include("src/network_matrices_creation.jl")
include("src/network_simulation.jl")
using Random
N = 1000
eps = 0.05
a = 0.5
b = bmatrix(pi/2-0.1, eps)
σ = 0.0506
G = wattsstrogatzmatrix(N, 3, 1);#0.232)


In [23]:
# Set the initial conditions

x_0 = zeros(2*N)
x_0[2 .* (1:N) .- 1] = rand(N) .* 2 .* a .- a
x_0[2 .* (1:N)] = rand(N) .* 2 .* (-a + a^3 / 3) .- (-a + a^3 / 3) #rand(Uniform(-a + a^3 / 3, a - a^3 / 3), N)

# Define the ODE problem
prob = ODEProblem((dx, x, params, t) -> coupled_fhn_eom!(dx, x, params[1], params[2], params[3], G, b), x_0, (0.0, 300.0), [a, eps, σ])

# Solve the ODE problem
# alg = Tsit5()
sol = solve(prob);

### Plots

In [24]:
include("src/network_sol_analysis.jl")
using GLMakie
using LaTeXStrings

f = Figure(size = (800, 600))
ax = Axis(f[1, 1])
ax.title = "Kuramoto Order Parameter"
ax.xlabel = "Time"
ax.ylabel = "Kuramoto Order Parameter"
t_val, kuramoto_val = kuramoto_time_series(sol, N)
lines!(ax, t_val, kuramoto_val)
f


## Study Cluster Synchronization in the integration of the FHN model.

### First, a test

Based on what is shown in the paper (https://doi.org/10.48550/arXiv.2303.08668), first we test with a complete graph.

In [4]:
include("src/msf.jl")
zero_msf = msf_zero()
println("zero_msf = ", zero_msf)
println("msf = ", master_stability_function(zero_msf, 0))

zero_msf = 0.20373021436669891
msf = 

-1.8873508449642648e-17


In [5]:
include("src/network_matrices_creation.jl")
include("src/network_simulation.jl")
include("src/cluster_synch.jl")
using Random
N = 10
eps = 0.05
a = 0.5
b = bmatrix(pi/2-0.1, eps)
G = test_matrix_for_cluster_synch();
eigenvalues, eigenvectors, clusters, s_matrices = s_matrix_method(G)
eigenvalues = unique(round.(eigenvalues, digits=8))[2:end]
critical_couplings = zero_msf./eigenvalues #unique(zero_msf./eigenvalues)[2:end]#
println("Critical Couplings: ", critical_couplings)
println("Eigenvalues: ", eigenvalues)
σ = critical_couplings[2]*1.1


Critical Couplings: [0.020373021436669893, 0.011984130256864641, 0.009701438779366614]
Eigenvalues: [10.0, 17.0, 21.0]


0.013182543282551107

In [6]:
x_0 = zeros(2*N)
# x_0[2 .* (1:N) .- 1] = rand(N) .* 2 .* a .- a
# x_0[2 .* (1:N)] = rand(N) .* 2 .* (-a + a^3 / 3) .- (-a + a^3 / 3)

prob = ODEProblem((dx, x, params, t) -> coupled_fhn_eom!(dx, x, params[1], params[2], params[3], G, b), x_0, (0.0, 1000.0), [a, eps, σ])

sol = solve(prob; dtmax=0.5);


In [7]:
using GLMakie
include("src/cluster_synch.jl")
include("src/network_sol_analysis.jl")

colors = ["#5ec962", "#21918c", "#3b528b", :red]
f = Figure(size= (1000, 600))
uni_clusters = unique(clusters)
for (i, cluster) in enumerate(uni_clusters)
    if isempty(cluster)
        continue
    else
        if i .< length(uni_clusters) && !isempty(uni_clusters[i+1])
            cluster = setdiff(cluster[1], uni_clusters[i+1][1])
        else
            cluster = cluster[1]
        end
        println("Cluster $i: $cluster")

        ax = Axis(f[i, 1])
        ax.xlabel = "Time"
        ax.ylabel = "Synch Error"
        t_values, synch_error = local_synch_error(sol, cluster)
        lines!(ax, t_values, synch_error; label="Cluster $cluster", linewidth=1, color = colors[i])
        axislegend()
    end
end
display(f);

Cluster 1: [1, 2, 3]


Cluster 2: [4, 5, 6]
Cluster 3: [7, 8, 9, 10]


In [8]:
using GLMakie
using Trapz
using ProgressMeter
using Base.Threads

include("src/cluster_synch.jl")
include("src/network_sol_analysis.jl")

N_d = 10
N_realizations = 10
d_sweep = range(0.0, 0.025, length=N_d)
t_measurement = 500.0
t_transient = 500.0
cluster_synch_averages = zeros(N_d, 3)
global_synch_averages = zeros(N_d)

f = Figure(size= (800, 600))
uni_clusters = unique(clusters)
@showprogress for (k, σ) in enumerate(d_sweep)
    temp = zeros(N_realizations, 3)
    for realization in 1:N_realizations
        x_0 = zeros(2*N) + 0.1 .* randn(2*N)
        prob = ODEProblem((dx, x, params, t) -> coupled_fhn_eom!(dx, x, params[1], params[2], params[3], G, b), x_0, (0.0, t_transient + t_measurement), [a, eps, σ])
        # alg = Tsit5()
        sol = solve(prob; dtmax=0.5)
        t_val, global_synch = synch_error_time_series(sol)
        global_synch_averages[k] += trapz(t_val[findfirst(t_val .> t_transient):end], global_synch[findfirst(t_val .> t_transient):end])/(t_measurement * N_realizations)
        for (i, cluster) in enumerate(uni_clusters)
            if isempty(cluster)
                continue
            else
                if i .< length(uni_clusters) && !isempty(uni_clusters[i+1])
                    cluster = setdiff(cluster[1], uni_clusters[i+1][1])
                else
                    cluster = cluster[1]
                end
                t_values, synch_error = local_synch_error(sol, cluster)
                # cluster_synch_averages[k, i] += trapz(t_values[findfirst(t_values .> t_transient):end], synch_error[findfirst(t_values .> t_transient):end])/(t_measurement * N_realizations)
                temp[realization, i] = trapz(t_values[findfirst(t_values .> t_transient):end], synch_error[findfirst(t_values .> t_transient):end])/(t_measurement * N_realizations)
            end
        end
    end
    cluster_synch_averages[k, :] = sum(temp, dims=1)
end

ax = Axis(f[1, 1])
ax.xlabel = "Coupling Strength"
ax.ylabel = "Average Synch Error"
for i in 1:3
    cluster = uni_clusters[i]
    if i .< length(uni_clusters) && !isempty(uni_clusters[i+1])
        cluster = setdiff(cluster[1], uni_clusters[i+1][1])
    else
        cluster = cluster[1]
    end
    lines!(ax, d_sweep, cluster_synch_averages[:, i]; label="Cluster $cluster", linewidth=1)
end
lines!(ax, d_sweep, global_synch_averages; label="Global", linestyle=:dash)
vlines!(ax, critical_couplings; label="Critical Couplings", linewidth=1, color = :red)
axislegend()
display(f);


Progress:  20%|████████▎                                |  ETA: 0:00:32

Progress:  30%|████████████▎                            |  ETA: 0:00:29

Progress:  40%|████████████████▍                        |  ETA: 0:00:24

Progress:  50%|████████████████████▌                    |  ETA: 0:00:19

Progress:  60%|████████████████████████▋                |  ETA: 0:00:15

Progress:  70%|████████████████████████████▊            |  ETA: 0:00:11

Progress:  80%|████████████████████████████████▊        |  ETA: 0:00:07

Progress:  90%|████████████████████████████████████▉    |  ETA: 0:00:03

Progress: 100%|█████████████████████████████████████████| Time: 0:00:33


In [9]:
include("src/network_sol_analysis.jl")
using GLMakie
using LaTeXStrings

f = Figure(size = (800, 200))
ax = Axis(f[1, 1])
ax.title = "Kuramoto Order Parameter"
ax.xlabel = "Time"
ax.ylabel = "Kuramoto Order Parameter"
t_val, kuramoto_val = kuramoto_time_series(sol, N)
lines!(ax, t_val, kuramoto_val)
f

### Now for a watts strogatz network:

In [10]:
include("src/network_sol_analysis.jl")
using GLMakie
using LaTeXStrings

f = Figure(size = (800, 300))
ax = Axis(f[1, 1])
ax.title = "Kuramoto Order Parameter"
ax.xlabel = "Time"
ax.ylabel = "Kuramoto Order Parameter"
t_val, kuramoto_val = kuramoto_time_series(sol, N)
lines!(ax, t_val, kuramoto_val)
f

ErrorException: cannot assign a value to imported variable Graphs.adjacency_matrix from module Main